In [ ]:
#| default_exp core

# core

> Finding magnetic discontinuities

In [ ]:
#| hide 
from nbdev.showdoc import *

In [ ]:
#| code-summary: import all the packages needed for the project
#| export

from fastcore.utils import *
from fastcore.test import *
from ids_finder.utils import *
import polars as pl
import polars.selectors as cs
import xarray as xr

try:
    import modin.pandas as pd
    from modin.config import ProgressBar
    ProgressBar.enable()
except ImportError:
    import pandas as pd

import pandas
    
import numpy as np
from xarray_einstats import linalg

from datetime import timedelta

from loguru import logger

import pytplot
from pytplot import timebar, store_data, tplot, split_vec, join_vec, tplot_options, options, highlight, degap

import pdpipe as pdp
from multipledispatch import dispatch

from collections.abc import Callable
from pandas import (
    DataFrame,
    Timestamp,
)
from xarray.core.dataarray import DataArray

## ID identification

The first index is $$ \frac{\sigma(B)}{Max(\sigma(B_-),\sigma(B_+))} $$
The second index is $$ \frac{\sigma(B_- + B_+)} {\sigma(B_-) + \sigma(B_+)} $$
The ﬁrst two conditions guarantee that the ﬁeld changes of the IDs identiﬁed are large enough to be distinguished from the stochastic ﬂuctuations on magnetic ﬁelds, while the third is a supplementary condition toreduce the uncertainty of recognition.

third index (relative field jump) is $$ \frac{| \Delta \vec{B} |}{|B_{bg}|} $$ a supplementary condition toreduce the uncertainty of recognition

### Index of the standard deviation

In [ ]:
help(compute_index_std)

Help on function compute_index_std in module ids_finder.utils:

compute_index_std(data, tau)
    helper function to compute standard deviation index



### Index of fluctuation

In [ ]:
help(compute_index_fluctuation)
help(compute_index_fluctuation_xr)

Help on function compute_index_fluctuation in module ids_finder.utils:

compute_index_fluctuation(data, tau)
    helper function to compute fluctuation index
    
    Notes: the results returned are a little bit different for the two implementations (because of the implementation of `std`).

Help on function compute_index_fluctuation_xr in module ids_finder.utils:

compute_index_fluctuation_xr(data: xarray.core.dataarray.DataArray, tau: int) -> xarray.core.dataarray.DataArray
    Computes the fluctuation index for a given data array based on a specified time interval.
    
    Parameters:
    - data: The xarray DataArray containing the data to be processed.
    - tau: Time interval in seconds for resampling.
    
    Returns:
    - fluctuation: xarray DataArray containing the fluctuation indices.
    
    Notes
    -----
        ddof=0 is used for calculating the standard deviation. (ddof=1 is for sample standard deviation)



### Index of the relative field jump

## Plotting

In [ ]:
#| export

@dispatch(object, xr.DataArray)
def get_candidate_data_xr(candidate, data, coord:str=None, neighbor:int=0) -> xr.DataArray:
    duration = candidate['tstop'] - candidate['tstart']
    offset = neighbor*duration
    temp_tstart = candidate['tstart'] - offset
    temp_tstop = candidate['tstop'] + offset
    
    return data.sel(time=slice(temp_tstart,  temp_tstop))

@dispatch(object, pl.DataFrame)
def get_candidate_data_pl(candidate, data, coord:str=None, neighbor:int=0) -> xr.DataArray:
    """
    Notes
    -----
    much slower than `get_candidate_data_xr`
    """
    duration = candidate['tstart'] - candidate['tstop']
    offset = neighbor*duration
    temp_tstart = candidate['tstart'] - offset
    temp_tstop = candidate['tstop'] + offset
    
    temp_data = data.filter(
        pl.col("time").is_between(temp_tstart, temp_tstop)
    )
    
    dims = ["v_dim", "time"]
    coords = {
        "time": temp_data['time'], 
        "v_dim": ["BX", "BY", "BZ"]
        }
    return xr.DataArray([ temp_data['BX'], temp_data['BY'], temp_data['BZ']], dims=dims, coords=coords)

def get_candidate_data(candidate, data, coord:str=None, neighbor:int=0) -> xr.DataArray:
    if isinstance(data, xr.DataArray):
        return get_candidate_data_xr(candidate, data, coord=coord, neighbor=neighbor)
    elif isinstance(data, pl.DataFrame):    
        return get_candidate_data_pl(candidate, data, coord=coord, neighbor=neighbor)

def get_candidates(candidates: DataFrame, candidate_type=None, num:int=4):
    
    if candidate_type is not None:
        _candidates = candidates[candidates['type'] == candidate_type]
    else:
        _candidates = candidates
    
    # Sample a specific number of candidates if num is provided and it's less than the total number
    if num < len(_candidates):
        logger.info(f"Sampling {num} {candidate_type} candidates out of {len(_candidates)}")
        return _candidates.sample(num)
    else:
        return _candidates

In [ ]:
#| export
from pyspedas.cotrans import minvar_matrix_make
from pyspedas import tvector_rotate

In [ ]:
#| export
def plot_basic(
    data, tstart, tstop, tau, mva_tstart=None, mva_tstop=None, neighbor: int = 1
):
    if mva_tstart is None:
        mva_tstart = tstart
    if mva_tstop is None:
        mva_tstop = tstop

    mva_b = data.sel(time=slice(mva_tstart, mva_tstop))
    store_data("fgm", data={"x": mva_b.time, "y": mva_b})
    minvar_matrix_make("fgm")  # get the MVA matrix

    temp_tstart = pd.Timestamp(tstart) - pd.Timedelta(neighbor * tau, unit="s")
    temp_tstop = pd.Timestamp(tstop) + pd.Timedelta(neighbor * tau, unit="s")

    temp_b = data.sel(time=slice(temp_tstart, temp_tstop))
    store_data("fgm", data={"x": temp_b.time, "y": temp_b})
    temp_btotal = calc_vec_mag(temp_b)
    store_data("fgm_btotal", data={"x": temp_btotal.time, "y": temp_btotal})

    tvector_rotate("fgm_mva_mat", "fgm")
    split_vec("fgm_rot")
    pytplot.data_quants["fgm_btotal"]["time"] = pytplot.data_quants["fgm_rot"][
        "time"
    ]  # NOTE: whenever using `get_data`, we may lose precision in the time values. This is a workaround.
    join_vec(
        [
            "fgm_rot_x",
            "fgm_rot_y",
            "fgm_rot_z",
            "fgm_btotal",
        ],
        new_tvar="fgm_all",
    )

    options("fgm", "legend_names", [r"$B_x$", r"$B_y$", r"$B_z$"])
    options("fgm_all", "legend_names", [r"$B_l$", r"$B_m$", r"$B_n$", r"$B_{total}$"])
    options("fgm_all", "ysubtitle", "[nT LMN]")
    highlight(["fgm", "fgm_all"], [tstart.timestamp(), tstop.timestamp()])
    degap("fgm")
    degap("fgm_all")

def format_candidate_title(candidate: pandas.Series):
    format_float = lambda x: rf"$\bf {x:.2f} $" if isinstance(x, (float, int)) else rf"$\bf {x} $"

    base_line = rf'$\bf {candidate.get("type", "N/A")} $ candidate (time: {candidate.get("time", "N/A")}) with index '
    index_line = rf'i1: {format_float(candidate.get("index_std", "N/A"))}, i2: {format_float(candidate.get("index_fluctuation", "N/A"))}, i3: {format_float(candidate.get("index_diff", "N/A"))}'
    info_line = rf'$B_n/B$: {format_float(candidate.get("BnOverB", "N/A"))}, $dB/B$: {format_float(candidate.get("dBOverB", "N/A"))}, $(dB/B)_{{max}}$: {format_float(candidate.get("dBOverB_max", "N/A"))},  $Q_{{mva}}$: {format_float(candidate.get("Q_mva", "N/A"))}'
    title = rf"""{base_line}
    {index_line}
    {info_line}"""
    return title


def plot_candidate(candidate: pandas.Series):
    if pd.notnull(candidate.get("d_tstart")) and pd.notnull(candidate.get("d_tstop")):
        plot_basic(
            sat_fgm,
            candidate["tstart"],
            candidate["tstop"],
            tau,
            candidate["d_tstart"],
            candidate["d_tstop"],
        )
    else:
        plot_basic(sat_fgm, candidate["tstart"], candidate["tstop"], tau)

    tplot_options("title", format_candidate_title(candidate))

    if "d_time" in candidate.keys():
        timebar(candidate["d_time"].timestamp(), color="red")
    if "d_tstart" in candidate.keys() and not pd.isnull(candidate["d_tstart"]):
        timebar(candidate["d_tstart"].timestamp())
    if "d_tstop" in candidate.keys() and not pd.isnull(candidate["d_tstop"]):
        timebar(candidate["d_tstop"].timestamp())

    # tplot(['fgm','fgm_all'])
    tplot("fgm_all")


def plot_candidates(
    candidates: pandas.DataFrame, candidate_type=None, num=4, plot_func=plot_candidate
):
    """Plot a set of candidates.

    Parameters:
    - candidates (pd.DataFrame): DataFrame containing the candidates.
    - candidate_type (str, optional): Filter candidates based on a specific type.
    - num (int): Number of candidates to plot, selected randomly.
    - plot_func (callable): Function used to plot an individual candidate.

    """

    # Filter by candidate_type if provided
    candidates = get_candidates(candidates, candidate_type, num)

    # Plot each candidate using the provided plotting function
    for _, candidate in candidates.iterrows():
        plot_func(candidate)

## ID parameters

### Duration

Definitions of duration
- Define $d^* = \max( | dB / dt | ) $, and then define time interval where $| dB/dt |$ decreases to $d^*/4$

In [ ]:
#| export
THRESHOLD_RATIO  = 1/4

from typing import Tuple

def calc_duration(vec: xr.DataArray, threshold_ratio=THRESHOLD_RATIO) -> pandas.Series:
    # NOTE: gradient calculated at the edge is not reliable.
    vec_diff = vec.differentiate("time", datetime_unit="s").isel(time=slice(1,-1))
    vec_diff_mag = linalg.norm(vec_diff, dims='v_dim')

    # Determine d_star based on trend
    if vec_diff_mag.isnull().all():
        raise ValueError("The differentiated vector magnitude contains only NaN values. Cannot compute duration.")
    
    d_star_index = vec_diff_mag.argmax(dim="time")
    d_star = vec_diff_mag[d_star_index]
    d_time = vec_diff_mag.time[d_star_index]
    
    threshold = d_star * threshold_ratio

    start_time, end_time = find_start_end_times(vec_diff_mag, d_time, threshold)

    return pandas.Series({
        'd_star': d_star.item(),
        'd_time': d_time.values,
        'threshold': threshold.item(),
        'd_tstart': start_time,
        'd_tstop': end_time,
    })

def calc_d_duration(vec: xr.DataArray, d_time, threshold) -> pd.Series:
    vec_diff = vec.differentiate("time", datetime_unit="s")
    vec_diff_mag = linalg.norm(vec_diff, dims='v_dim')

    start_time, end_time = find_start_end_times(vec_diff_mag, d_time, threshold)

    return pandas.Series({
        'd_tstart': start_time,
        'd_tstop': end_time,
    })
 
def find_start_end_times(vec_diff_mag: xr.DataArray, d_time, threshold) -> Tuple[pd.Timestamp, pd.Timestamp]:
    # Determine start time
    pre_vec_mag = vec_diff_mag.sel(time=slice(None, d_time))
    start_time = get_time_from_condition(pre_vec_mag, threshold, "last_below")

    # Determine stop time
    post_vec_mag = vec_diff_mag.sel(time=slice(d_time, None))
    end_time = get_time_from_condition(post_vec_mag, threshold, "first_below")

    return start_time, end_time


def get_time_from_condition(vec: xr.DataArray, threshold, condition_type) -> pd.Timestamp:
    if condition_type == "first_below":
        condition = vec < threshold
        index_choice = 0
    elif condition_type == "last_below":
        condition = vec < threshold
        index_choice = -1
    else:
        raise ValueError(f"Unknown condition_type: {condition_type}")

    where_result = np.where(condition)[0]

    if len(where_result) > 0:
        return vec.time[where_result[index_choice]].values
    return None

In [ ]:
#| export
def calc_candidate_duration(candidate: pd.Series, data, get_candidate_data_fn:Callable =get_candidate_data_xr) -> pd.Series:
    try:
        candidate_data = get_candidate_data_fn(candidate, data)
        return calc_duration(candidate_data)
    except Exception as e:
        # logger.debug(f"Error for candidate {candidate} at {candidate['time']}: {str(e)}") # can not be serialized
        print(f"Error for candidate {candidate} at {candidate['time']}: {str(e)}")
        raise e

def calc_candidate_d_duration(candidate, data , get_candidate_data_fn:Callable =get_candidate_data) -> pd.Series:
    try:
        if pd.isnull(candidate['d_tstart']) or pd.isnull(candidate['d_tstop']):
            candidate_data = get_candidate_data_fn(candidate, data, neighbor=1)
            d_time = candidate['d_time']
            threshold = candidate['threshold']
            return calc_d_duration(candidate_data, d_time, threshold)
        else:
            return pandas.Series({
                'd_tstart': candidate['d_tstart'],
                'd_tstop': candidate['d_tstop'],
            })
    except Exception as e:
        # logger.debug(f"Error for candidate {candidate} at {candidate['time']}: {str(e)}")
        print(f"Error for candidate {candidate} at {candidate['time']}: {str(e)}")
        raise e

def calibrate_candidate_duration(
    candidate: pd.Series, data:xr.DataArray, data_resolution, ratio = 3/4
):
    """
    Calibrates the candidate duration. 
    - If only one of 'd_tstart' or 'd_tstop' is provided, calculates the missing one based on the provided one and 'd_time'.
    - Then if this is not enough points between 'd_tstart' and 'd_tstop', returns None for both.
    
    
    Parameters
    ----------
    - candidate (pd.Series): The input candidate with potential missing 'd_tstart' or 'd_tstop'.
    
    Returns
    -------
    - pd.Series: The calibrated candidate.
    """
    
    start_notnull = pd.notnull(candidate['d_tstart'])
    stop_notnull = pd.notnull(candidate['d_tstop']) 
    
    match start_notnull, stop_notnull:
        case (True, True):
            d_tstart = candidate['d_tstart']
            d_tstop = candidate['d_tstop']
        case (True, False):
            d_tstart = candidate['d_tstart']
            d_tstop = candidate['d_time'] -  candidate['d_tstart'] + candidate['d_time']
        case (False, True):
            d_tstart = candidate['d_time'] -  candidate['d_tstop'] + candidate['d_time']
            d_tstop = candidate['d_tstop']
        case (False, False):
            return pandas.Series({
                'd_tstart': None,
                'd_tstop': None,
            })
    
    duration = d_tstop - d_tstart
    num_of_points_between = data.time.sel(time=slice(d_tstart, d_tstop)).count().item()
    
    if num_of_points_between <= (duration/data_resolution) * ratio:
        d_tstart = None
        d_tstop = None
    
    return pandas.Series({
        'd_tstart': d_tstart,
        'd_tstop': d_tstop,
    })

### ID classification

In this method, TDs and RDs satisfy $ \frac{ |B_N| }{ |B_{bg}| } < 0.2$ and $ | \frac{ \Delta |B| }{ |B_{bg}| } | > 0.4$ B BN bg ∣∣ ∣∣ , < D 0.2 B B bg ∣∣ ∣ ∣ , respectively. Moreover, IDs with < 0.4 B BN bg ∣∣ ∣∣ , < D 0.2 B B bg ∣∣ ∣ ∣ could be either TDs or RDs, and so are termed EDs. Similarly, NDs are defined as > 0.4 B BN bg ∣∣ ∣∣ , > D 0.2 B B bg ∣∣ ∣ ∣ because they can be neither TDs nor RDs. It is worth noting that EDs and NDs here are not physical concepts like RDs and TDs. RDs or TDs correspond to specific types of structures in the MHD framework, while EDs and NDs are introduced just to better quantify the statistical results.


Criteria Used to Classify Discontinuities on the Basis of Magnetic Data Type

| Type   |  $\|B_n/B\|$      | $\| \Delta B / B \|$  |
|----------|-------------|------|
| Rotational (RD) | large | small |
| Tangential (TD) | small |  large |
| Either (ED) | small | small |
| Neither (ND) | large | large |


#### minimum variance analysis (MVA)

To ensure the accuracy of MVA, only when the ratio of the middle to the minimum eigenvalue (labeled QMVA for simplicity) is larger than 3 are the results used for further analysis.

In [ ]:
#| export
BnOverB_RD_lower_threshold = 0.4
dBOverB_RD_upper_threshold = 0.2

BnOverB_TD_upper_threshold = 0.2
dBOverB_TD_lower_threshold = dBOverB_RD_upper_threshold

BnOverB_ED_upper_threshold = BnOverB_RD_lower_threshold
dBOverB_ED_upper_threshold = dBOverB_TD_lower_threshold

BnOverB_ND_lower_threshold = BnOverB_TD_upper_threshold
dBOverB_ND_lower_threshold = dBOverB_RD_upper_threshold


In [ ]:
#| export
from pyspedas.cotrans.minvar import minvar

In [ ]:
#| export
def calc_classification_index(data: xr.DataArray):
    
    vrot, v, w = minvar(data.to_numpy()) # NOTE: using `.to_numpy()` will significantly speed up the computation.
    Vl = v[:,0] # Maximum variance direction eigenvector

    B_rot = xr.DataArray(vrot, dims=['time', 'v_dim'], coords={'time': data.time})
    B = calc_vec_mag(B_rot)
    B_n = B_rot.isel(v_dim=2)
    
    B_mean = B.mean(dim="time")
    B_n_mean = B_n.mean(dim="time")
    
    BnOverB = B_n_mean / B_mean
    # BnOverB = np.abs(B_n / B).mean(dim="time")

    dB = B.isel(time=-1) - B.isel(time=0)
    dBOverB = np.abs(dB / B_mean)
    dBOverB_max = (B.max(dim="time") - B.min(dim="time")) / B_mean
    
    
    return pandas.Series({
        'Vl_x': Vl[0],
        'Vl_y': Vl[1],
        'Vl_z': Vl[2],
        'eig0': w[0],
        'eig1': w[1],
        'eig2': w[2],
        'Q_mva': w[1]/w[2],
        'B': B_mean.item(),
        'B_n': B_n_mean.item(),
        'dB': dB.item(),
        'BnOverB': BnOverB.item(), 
        'dBOverB': dBOverB.item(),
        'dBOverB_max': dBOverB_max.item(),
        })

In [ ]:
#| export
def classify_id(BnOverB, dBOverB):
    BnOverB = np.abs(np.asarray(BnOverB))
    dBOverB = np.asarray(dBOverB)

    s1 = (BnOverB > BnOverB_RD_lower_threshold)
    s2 = (dBOverB > dBOverB_RD_upper_threshold)
    s3 = (BnOverB > BnOverB_TD_upper_threshold)
    s4 = s2 # note: s4 = (dBOverB > dBOverB_TD_lower_threshold)
    
    RD = s1 & ~s2
    TD = ~s3 & s4
    ED = ~s1 & ~s4
    ND = s3 & s2

    # Create an empty result array with the same shape
    result = np.empty_like(BnOverB, dtype=object)

    result[RD] = "RD"
    result[TD] = "TD"
    result[ED] = "ED"
    result[ND] = "ND"

    return result


### Field rotation angles
The PDF of the field rotation angles across the solar-wind IDs is well fitted by the exponential function exp(−θ/)...

In [ ]:
#| export
def calc_rotation_angle(v1, v2):
    """
    Computes the rotation angle between two vectors.
    
    Parameters:
    - v1: The first vector.
    - v2: The second vector.
    """
    
    if v1.shape != v2.shape:
        raise ValueError("Vectors must have the same shape.")

    # convert xr.Dataarray to numpy arrays
    if isinstance(v1, DataArray):
        v1 = v1.to_numpy()
    if isinstance(v2, DataArray):
        v2 = v2.to_numpy()
    
    # Normalize the vectors
    v1_u = v1 / np.linalg.norm(v1, axis=-1, keepdims=True)
    v2_u = v2 / np.linalg.norm(v2, axis=-1, keepdims=True)
    
    # Calculate the cosine of the angle for each time step
    cosine_angle = np.sum(v1_u * v2_u, axis=-1)
    
    # Clip the values to handle potential floating point errors
    cosine_angle = np.clip(cosine_angle, -1, 1)
    
    angle = np.arccos(cosine_angle)
    
    # Convert the angles from radians to degrees
    return np.degrees(angle)

def calc_candidate_rotation_angle(candidates, data:  xr.DataArray):
    """
    Computes the rotation angle(s) at two different time steps.
    """
    
    tstart = candidates['d_tstart']
    tstop = candidates['d_tstop']
    
    # Convert Series to numpy arrays if necessary
    if isinstance(tstart, pd.Series):
        tstart = tstart.to_numpy()
        tstop = tstop.to_numpy()
        # no need to Handle NaT values (as `calibrate_candidate_duration` will handle this)
    
    # Get the vectors at the two time steps
    vecs_before = data.sel(time=tstart, method="nearest")
    vecs_after = data.sel(time=tstop, method="nearest")
    
    # Compute the rotation angle(s)
    rotation_angles = calc_rotation_angle(vecs_before, vecs_after)
    return rotation_angles

### Assign satellite locations to the discontinuities

In [ ]:
#| export
def get_candidate_location(candidate, location_data: DataArray):
    return location_data.sel(time = candidate['d_time']).to_series()

## Processing the whole dataset

In [ ]:
#| export
def get_ID_filter_condition(
    index_std_threshold = 2,
    index_fluc_threshold = 1,
    index_diff_threshold = 0.1,
    sparse_num = 15
):
    return (
        (pl.col("index_std") > index_std_threshold)
        & (pl.col("index_fluctuation") > index_fluc_threshold)
        & (pl.col("index_diff") > index_diff_threshold)
        & (
            pl.col("index_std").is_finite()
        )  # for cases where neighboring groups have std=0
        & (
            pl.col("count") > sparse_num
        )  # filter out sparse intervals, which may give unreasonable results.
    )



In [ ]:
#| export
from pdpipe.util import out_of_place_col_insert

In [ ]:
#| code-summary: patch `pdp.ApplyToRows` to work with `modin` DataFrames
#| export

@patch
def _transform(self: pdp.ApplyToRows, X, verbose):
    new_cols = X.apply(self._func, axis=1)
    if isinstance(new_cols, (pd.Series, pandas.Series)):
        loc = len(X.columns)
        if self._follow_column:
            loc = X.columns.get_loc(self._follow_column) + 1
        return out_of_place_col_insert(
            X=X, series=new_cols, loc=loc, column_name=self._colname
        )
    if isinstance(new_cols, (pd.DataFrame, pandas.DataFrame)):
        sorted_cols = sorted(list(new_cols.columns))
        new_cols = new_cols[sorted_cols]
        if self._follow_column:
            inter_X = X
            loc = X.columns.get_loc(self._follow_column) + 1
            for colname in new_cols.columns:
                inter_X = out_of_place_col_insert(
                    X=inter_X,
                    series=new_cols[colname],
                    loc=loc,
                    column_name=colname,
                )
                loc += 1
            return inter_X
        assign_map = {
            colname: new_cols[colname] for colname in new_cols.columns
        }
        return X.assign(**assign_map)
    raise TypeError(  # pragma: no cover
        "Unexpected type generated by applying a function to a DataFrame."
        " Only Series and DataFrame are allowed."
    )

In [ ]:
#| export
def calc_candidate_classification_index(candidate, data):
    return calc_classification_index(
        data.sel(time=slice(candidate["d_tstart"], candidate["d_tstop"]))
    )

In [ ]:
#| export
def convert_to_dataframe(
    data: pl.DataFrame, # orignal Dataframe
):
    "convert data into a pandas/modin DataFrame"
    if isinstance(data, pl.LazyFrame):
        data = data.collect().to_pandas(use_pyarrow_extension_array=True)
    if isinstance(data, pl.DataFrame):
        data = data.to_pandas(use_pyarrow_extension_array=True)
    if not isinstance(data, pd.DataFrame):  # `modin` supports
        data = pd.DataFrame(data)
    return data

In [ ]:
#| code-summary: Pipelines Class for processing IDs
#| export
class IDsPipeline:
    def __init__(self):
        pass
    # fmt: off
    def calc_duration(self, sat_fgm: xr.DataArray):
        return pdp.PdPipeline([
            pdp.ApplyToRows(
                lambda candidate: calc_candidate_duration(candidate, sat_fgm),
                func_desc="calculating duration parameters"
            ),
            pdp.ApplyToRows(
                lambda candidate: calc_candidate_d_duration(candidate, sat_fgm),
                func_desc="calculating duration parameters if needed"
            ),
        ])

    def calibrate_duration(self, sat_fgm, data_resolution):
        return \
            pdp.ApplyToRows(
                lambda candidate: calibrate_candidate_duration(candidate, sat_fgm, data_resolution),
                func_desc="calibrating duration parameters if needed"
            )

    def classify_id(self, sat_fgm):
        return pdp.PdPipeline([
            pdp.ApplyToRows(
                lambda candidate: calc_candidate_classification_index(candidate, sat_fgm),
                func_desc='calculating index "q_mva", "BnOverB" and "dBOverB"'
            ),
            pdp.ColByFrameFunc(
                "type",
                lambda df: classify_id(df["BnOverB"], df["dBOverB"]),
                func_desc="classifying the type of the ID"
            ),
        ])
    
    def calc_rotation_angle(self, sat_fgm):
        return \
            pdp.ColByFrameFunc(
                "rotation_angle",
                lambda df: calc_candidate_rotation_angle(df, sat_fgm),
                func_desc="calculating rotation angle",
            ) 

    def assign_coordinates(self, sat_state):
        return \
            pdp.ApplyToRows(
                lambda candidate: get_candidate_location(candidate, sat_state),
                func_desc="assigning coordinates",
            )
    # fmt: on
    # ... you can add more methods as needed

In [ ]:
#| export
def process_candidates(
    candidates: pl.DataFrame, 
    sat_fgm: xr.DataArray, 
    sat_state: xr.DataArray,
    data_resolution: timedelta
):
    
    id_pipelines = IDsPipeline()

    candidates = id_pipelines.calc_duration(sat_fgm).apply(candidates)

    # calibrate duration
    temp_candidates = candidates.loc[
        lambda df: df["d_tstart"].isnull() | df["d_tstop"].isnull()
    ]
    if not temp_candidates.empty:
        candidates.update(
            id_pipelines.calibrate_duration(sat_fgm, data_resolution).apply(temp_candidates)
        )

    candidates = candidates.dropna()  # Remove candidates with NaN values
    
    ids = (
        id_pipelines.classify_id(sat_fgm) + 
        id_pipelines.calc_rotation_angle(sat_fgm) +
        id_pipelines.assign_coordinates(sat_state)
    ).apply(candidates)

    return ids

## Test case

In [ ]:
sat = 'jno'
coord = 'se'
tau = timedelta(seconds=60)
data_resolution = timedelta(seconds=1)

if True:
    year = 2011
    files = f'../data/{sat}_{year}.parquet'
    output = f'../data/{sat}_candidates_{year}_tau_{tau.seconds}.parquet'

    data = pl.scan_parquet(files).set_sorted('time').collect()
    sat_fgm = df2ts(data, ["BX", "BY", "BZ"], attrs={"coordinate_system": coord, "units": "nT"})
    sat_state = df2ts(data, ["X", "Y", "Z"], attrs={"coordinate_system": coord, "units": "km"})

    indices = compute_indices(data, tau)
    # filter condition
    sparse_num = tau / data_resolution // 3
    filter_condition = get_ID_filter_condition(sparse_num = sparse_num)

    candidates = indices.filter(filter_condition).with_columns(pl_format_time(tau))
    
    ids = process_candidates(candidates, sat_fgm, sat_state, data_resolution)
    df = pandas.DataFrame(ids)
    # pandas.DataFrame(ids).to_parquet(output)



21-Sep-23 22:58:43: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.

Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
Estimated completion of line 17: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00


In [ ]:
# Test different libraries to parallelize the computation
test = True
if test:
    pdp_test = pdp.ApplyToRows(
        lambda candidate: calc_candidate_duration(candidate, sat_fgm),  # fast a little bit
        # lambda candidate: calc_duration(get_candidate_data_xr(candidate, jno_fgm)),
        # lambda candidate: calc_duration(jno_fgm.sel(time=slice(candidate['tstart'], candidate['tstop']))),
        func_desc="calculating duration parameters",
    )
    candidates_pd = candidates.to_pandas()
    candidates_modin = pd.DataFrame(candidates_pd)
    
    # ---
    # successful cases
    # ---
    # candidates_pd.apply(lambda candidate: calc_candidate_duration(candidate, jno_fgm), axis=1) # Standard case: 37+s secs
    # candidates_pd.swifter.apply(calc_candidate_duration, axis=1, data=jno_fgm) # this works with dask, 80 secs
    # candidates_pd.swifter.set_dask_scheduler(scheduler="threads").apply(calc_candidate_duration, axis=1, data=jno_fgm) # this works with dask, 60 secs
    # candidates_pd.mapply(lambda candidate: calc_candidate_duration(candidate, jno_fgm), axis=1) # this works, 8 secs # not work? `DataFrame' object has no attribute 'mapply'
    # candidates_modin.apply(lambda candidate: calc_candidate_duration(candidate, jno_fgm), axis=1) # this works with ray, 8 secs # NOTE: can not work with dask
    pdp_test(candidates_modin) # this works, 8 secs
    
    # ---
    # failed cases
    # ---
    # candidates_modin.apply(calc_candidate_duration, axis=1, data=jno_fgm) # AttributeError: 'DataFrame' object has no attribute 'sel'
    # pdp_test(candidates_modin) # TypeError: Unexpected type generated by applying a function to a DataFrame. Only Series and DataFrame are allowed.


21-Sep-23 23:06:55: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.

Distributing Dataframe: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00
Estimated completion of line 17: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00


### Cases

In [ ]:
# plot_candidates(candidates.loc[lambda _: _['time']=='2012-07-10 02:31:15'])
temp_trange = ['2012-07-15 03:44', '2012-07-15 03:47']
temp_data = sat_fgm.sel(time=slice(*temp_trange))
temp_data.plot.scatter(x='time', hue='v_dim')
# temp_data.resample(time=pd.Timedelta(tau, unit='s')).map(calc_vec_std)
compute_index_std(temp_data, tau)

#### Case: neighboring data is missing, causing the calculation of the standard deviation index to be Inf

In [ ]:
# Case: neighboring data is missing, causing the calculation of the standard deviation index to be Inf.
temp_trange = ['2012-07-10 02:30', '2012-07-10 02:32']
temp_data = sat_fgm.sel(time=slice(*temp_trange))
temp_data.plot.scatter(x='time', hue='v_dim')
# temp_data.resample(time=pd.Timedelta(tau, unit='s')).map(calc_vec_std)
compute_index_std(temp_data, tau)

#### Caveats

In [ ]:
plot_candidates(candidates)

##### NOTE: Not very accurate for waving magnetic field...

In [ ]:
temp_candidate = {'time': Timestamp('2012-05-01 00:39:12'),
 'tstart': Timestamp('2012-05-01 00:38:56'),
 'tstop': Timestamp('2012-05-01 00:39:28'),
 'i1': 2.891042053414383,
 'i2': 2.389699609352786,
 'i3': 1.3916002784658887,
 'd_star': 0.27143595,
 'd_time': Timestamp('2012-05-01 00:39:18.672000'),
 'd_tstart': Timestamp('2012-05-01 00:39:14.672000'),
 'd_tstop': Timestamp('2012-05-01 00:39:19.671000'),
}

plot_candidate(temp_candidate)

In [ ]:
data = get_candidate_data_xr(temp_candidate, neighbor=1)
vec_diff = data.differentiate("time", datetime_unit="s", edge_order=2).isel(time=slice(1,-1))
vec_diff_mag = linalg.norm(vec_diff, dims='v_dim')
vec_diff_mag.plot()

##### NOTE: Small threshold_ratio values will tend to make the duration longer if the duration can be determined.


In [ ]:
# Test different threshold ratios
threshold_ratios = [1/8, 1/4, 0.3, 1/3, 1/2]
for threshold_ratio in threshold_ratios:
    temp_candidate.update(calc_duration(get_candidate_data_xr(temp_candidate), threshold_ratio=threshold_ratio).to_dict())
    plot_candidate(temp_candidate)